## Explore nominations table

In [17]:
import sqlite3 as sql

conn = sql.connect("nominations.db")
schema = conn.execute("PRAGMA table_info(nominations);").fetchall()
first_ten = conn.execute("SELECT * FROM nominations LIMIT 10;").fetchall()
for r in schema:
    print(r)
    
for r in first_ten:
    print(r)

OperationalError: no such table: nominations

## Create new table for ceremonies

In [3]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

conn.execute("DROP TABLE IF EXISTS ceremonies")
create_table = "CREATE TABLE ceremonies(id integer PRIMARY KEY, Year integer, Host text);"
conn.execute(create_table)

insert = "INSERT INTO ceremonies(Year, Host) VALUES (?,?);"
conn.executemany(insert, years_hosts)

print(conn.execute("PRAGMA table_info(ceremonies);").fetchall())
print(conn.execute("SELECT * FROM ceremonies LIMIT 10;").fetchall())

[(0, 'id', 'integer', 0, None, 1), (1, 'Year', 'integer', 0, None, 0), (2, 'Host', 'text', 0, None, 0)]
[(1, 2010, 'Steve Martin'), (2, 2009, 'Hugh Jackman'), (3, 2008, 'Jon Stewart'), (4, 2007, 'Ellen DeGeneres'), (5, 2006, 'Jon Stewart'), (6, 2005, 'Chris Rock'), (7, 2004, 'Billy Crystal'), (8, 2003, 'Steve Martin'), (9, 2002, 'Whoopi Goldberg'), (10, 2001, 'Steve Martin')]


## Foreign key constraints to ensure data integrity

In [ ]:
conn.execute("PRAGMA foreign_keys = ON;")

## Refactor nominations table to contain foreign key from ceremonies table

In [4]:
create_table = """
CREATE TABLE nominations_two(
id integer PRIMARY KEY,
category text,
nominee text,
movie text,
character text,
won integer,
ceremony_id integer,
FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id));
"""

copy_nominations = """
SELECT ceremonies.id, nominations.category,
nominations.nominee, nominations.movie, 
nominations.character , nominations.won 
FROM nominations
INNER JOIN ceremonies 
ON nominations.year == ceremonies.year
;
"""

insert_nominations_two = """
INSERT INTO nominations_two 
(ceremony_id, category, nominee, movie, character, won) 
VALUES (?,?,?,?,?,?);
"""

conn.execute("DROP TABLE IF EXISTS nominations_two")
conn.execute(create_table)
joined_nominations = conn.execute(copy_nominations).fetchall()
conn.executemany(insert_nominations_two, joined_nominations)
print(conn.execute("SELECT * FROM nominations_two LIMIT 5;").fetchall())

OperationalError: no such table: nominations

## Delete old nominations table

In [16]:
drop_nominations = "DROP TABLE nominations;"
conn.execute(drop_nominations)

rename_nominations_two = "ALTER TABLE nominations_two rename to nominations;"
conn.execute(rename_nominations_two)
conn.close()

OperationalError: no such table: nominations

## Create and population join table

In [ ]:
create_movies = "CREATE TABLE movies(id integer PRIMARY LEY,movie text);"
create_actors = "CREATE TABLE actors(id integer PRIMARY KEY,actor text);"
create_movies_actors = '''CREATE TABLE movies_actors(id integer PRIMARY KEY,
movie_id integer REFERENCES movies(id), actor_id INTEGER references actors(id));
'''
conn.execute(create_movies)
conn.execute(create_actors)
conn.execute(create_movies_actors)






insert_movies = "INSERT INTO movies (movie) SELECT DISTINCT movie from nominations;"
insert_actors = "INSERT INTO actors (actor) SELECT DISTINCT nominee from nominations;"
conn.execute(insert_movies)
conn.execute(insert_actors)

print(conn.execute("SELECT * FROM movies LIMIT 5;").fetchall())
print(conn.execute("SELECT * FROM actors LIMIT 5;").fetchall())

